In [2]:
from PattRecClasses import GaussD, HMM, MarkovChain
from matplotlib import pyplot as plt
import numpy as np


Load and clean the data

The moving average method was copied from https://stackoverflow.com/questions/14313510/how-to-calculate-rolling-moving-average-using-python-numpy-scipy 

In [3]:
#Load the data
import os 

data_dir = './data/training'
test_dir = './data/testing' 
dirs = ["/walk","/stand", "/run"]

data_stand = os.listdir(data_dir)


training_data = []
test_data = []

n = 6
for i in data_stand:
    #simple moving average
    data = np.loadtxt((data_dir+"/"+i), usecols=[2,3,4])
    ret = np.cumsum(data, dtype=float,axis=0)
    ret[n:] = ret[n:] - ret[:-n]
    data = ret[n - 1:] / n


    #REMOVE the first and last 500 datapoints
    training_data.append(data[500:-500,:])



for i in dirs:
    test_stand = os.listdir(test_dir + i)
    for j in test_stand:
        #simple moving average
        data = np.loadtxt((test_dir+i+"/"+j), usecols=[2,3,4])
        ret = np.cumsum(data, dtype=float,axis=0)
        ret[n:] = ret[n:] - ret[:-n]
        data = ret[n - 1:] / n

        #REMOVE the first and last 500 datapoints
        test_data.append((data[500:-500,:],i))


Setting up HMM model

In [4]:
#Set up Markovchain
nr_states = 3
mc = MarkovChain(np.array([1/nr_states, 1/nr_states, 1/nr_states]), np.array([[0.99, 0.005, 0.005], [0.005, 0.99, 0.005], [0.005, 0.005, 0.99]]))


#setting up output gaussian
cov1 = np.array([[1,0,0],[0,1,0], [0,0,1]])
cov2 = np.array([[1,0,0],[0,1,0], [0,0,1]])
cov3 = np.array([[1,0,0],[0,1,0], [0,0,1]])
cov12 = np.array([[1,0,0],[0,1,0], [0,0,1]])
cov22 = np.array([[1,0,0],[0,1,0], [0,0,1]])
cov32 = np.array([[1,0,0],[0,1,0], [0,0,1]])
g1 = GaussD(means=[0,-10,-0.1], cov=cov1)
g2 = GaussD(means=[0,-10,0.1], cov=cov2)
g3 = GaussD(means=[1,-10,-1], cov=cov3)
g12 = GaussD(means=[1,-9,1], cov=cov12)
g22 = GaussD(means=[1,-10,-3], cov=cov22)
g32 = GaussD(means=[-1,-20,-10], cov=cov32)

weights = [[0.9,0.1],[0.9, 0.1],[0.9, 0.1]]

#create HMM
hmm = HMM(mc, [[g1,g12],[g2, g22],[g3, g32]], weights)


Measure Accruacy before training

In [5]:
total_acc = 0
for r in range(len(test_data)):
    predict_seq = hmm.log_viterbi(test_data[r][0])
    seq_length = len(predict_seq)
    if test_data[r][1] == '/stand':
        print("Stand test")
        acc = np.sum(predict_seq == np.zeros(seq_length)) / seq_length
    elif test_data[r][1] == '/run':
        print("run test")
        acc = np.sum(predict_seq == np.full(seq_length,2)) / seq_length
    elif test_data[r][1] == '/walk':
        print("walk test")
        acc = np.sum(predict_seq == np.full(seq_length,1)) / seq_length
    total_acc += acc    
    print(acc)
print("total acc:", total_acc/len(test_data))

walk test
0.27219183845183004
walk test
0.29979879275653926
walk test
0.3112862369828314
Stand test
0.0
Stand test
1.0
Stand test
0.12805919180421171
run test
0.11581291759465479
run test
0.1456953642384106
run test
0.13177339901477833
total acc: 0.267179748982584


Train the model:

In [6]:
for i in range(1):
    print("epoch", i)
    hmm.train(training_data)

epoch 0


In [7]:
print("q: ", hmm.stateGen.q)
print("A: ",hmm.stateGen.A)
print("Weights: ", hmm.outoutWeights)
for dis_list in hmm.outputDistr:
    for dis in dis_list:
        print("mean: ",dis.means)
        print("cov ",dis.cov)

q:  [0.22402123 0.45147721 0.32450156]
A:  [[0.95334749 0.02482487 0.02182763]
 [0.02078476 0.95594843 0.02326681]
 [0.02571648 0.05191006 0.92237346]]
Weights:  [[0.3000590875166212, 0.6999409124833819], [0.7693192488218071, 0.2306807511782007], [0.4551311829478817, 0.5448688170521129]]
mean:  [-2.43740118 -8.93695179  0.52295716]
cov  [[0.00465728 0.         0.        ]
 [0.         0.01055272 0.        ]
 [0.         0.         0.00096757]]
mean:  [ 0.41753952 -2.25347711  1.04256794]
cov  [[0.00189498 0.         0.        ]
 [0.         0.00948893 0.        ]
 [0.         0.         0.00105663]]
mean:  [ -1.43103614 -10.78867661   1.17047773]
cov  [[0.0013923  0.         0.        ]
 [0.         0.00608844 0.        ]
 [0.         0.         0.0006193 ]]
mean:  [ 1.49748742 -9.17879169 -5.31510025]
cov  [[0.01126708 0.         0.        ]
 [0.         0.01195012 0.        ]
 [0.         0.         0.00214753]]
mean:  [  2.84030961 -10.9808666   -1.32005761]
cov  [[0.0013047  0.    

TEST Trained model


In [8]:
total_acc = 0
for r in range(len(test_data)):
    predict_seq = hmm.log_viterbi(test_data[r][0])
    seq_length = len(predict_seq)
    if test_data[r][1] == '/stand':
        print("Stand test")
        acc = np.sum(predict_seq == np.zeros(seq_length)) / seq_length
    elif test_data[r][1] == '/run':
        print("run test")
        acc = np.sum(predict_seq == np.full(seq_length,2)) / seq_length
    elif test_data[r][1] == '/walk':
        print("walk test")
        acc = np.sum(predict_seq == np.full(seq_length,1)) / seq_length
    total_acc += acc    
    print(acc)
print("total acc:", total_acc/len(test_data))

walk test
1.0
walk test
1.0
walk test
1.0
Stand test
0.0
Stand test
0.0
Stand test
0.0
run test
0.8950047725103404
run test
0.9417915650052283
run test
0.9735221674876847
total acc: 0.6455909450003615
